# Importing Packages

In [44]:
import os
import argparse
import cv2
import mediapipe as mp

# Unzip the Data

In [46]:
# importing the zipfile module 
from zipfile import ZipFile 
  
# loading the temp.zip and creating a zip object 
with ZipFile("./face-anonymizer-ptyhon-master.zip", 'r') as zObject: 
    # Extracting specific file in the zip 
    # into a specific location. 
    unzip_dir = './unzip'
    if not os.path.exists(unzip_dir):
        os.makedirs(unzip_dir)
    zObject.extractall(path="./output_dir") 
zObject.close() 

# Face Anonymyzer Function

In [117]:
def process_img(img, face_detection):

    H, W, _ = img.shape
    print(H, W, _)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = face_detection.process(img_rgb)

    if out.detections is not None:
        for detection in out.detections:
            
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box
            print(bbox)

            x1, y1, w, h = bbox.xmin, bbox.ymin, bbox.width, bbox.height

            x1 = int(x1*W)
            y1 = int(y1*H)
            w = int(w * W)
            h = int(h * H)

            print(x1, y1, w, h)

            # blur faces
            img[y1:y1 + h, x1:x1 + w, :] = cv2.blur(img[y1:y1 + h, x1:x1 + w, :], (30, 30))

    return img

# Argument Parser

In [118]:
args = argparse.ArgumentParser()

args.add_argument("--mode", default='image')
args.add_argument("--filePath", default='./unzip/Danish Ali.jpg')

args = args.parse_args(args=[])
print(args.mode)
print(args.filePath)

image
./unzip/Danish Ali.jpg


# Making Output Directory

In [119]:
output_dir = './output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Media Pipe ML Face Detection Package

In [120]:
mp_face_detection = mp.solutions.face_detection

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    
    if args.mode in ["image"]:
        # read image
        img = cv2.imread(args.filePath)

        img = process_img(img, face_detection)

        # save image
        cv2.imwrite(os.path.join(output_dir, 'output.png'), img)
        cv2.imshow('img',img)
        cv2.waitKey(0)

    elif args.mode in ['video']:

        cap = cv2.VideoCapture(args.filePath)
        ret, frame = cap.read()

        output_video = cv2.VideoWriter(os.path.join(output_dir, 'output.mp4'),
                                       cv2.VideoWriter_fourcc(*'MP4V'),
                                       25,
                                       (frame.shape[1], frame.shape[0]))

        while ret:

            frame = process_img(frame, face_detection)

            output_video.write(frame)

            ret, frame = cap.read()

        cap.release()
        output_video.release()


419 317 3
xmin: 0.2759319543838501
ymin: 0.2773107588291168
width: 0.4514608383178711
height: 0.3415416181087494

87 116 143 143
